# Radar Overlap Testing Notebook

Testing different solutions for parameter/value overlap issues.

In [2]:
# Import required libraries
import pandas as pd
import numpy as np
from mplsoccer import PyPizza, FontManager
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from PIL import Image
from scipy.stats import percentileofscore
import sys
import os
import matplotlib.font_manager as fm

# Add src directory to path
sys.path.append('src')
sys.path.append('./src')
sys.path.append(os.path.join(os.getcwd(), 'src'))

try:
    from enhanced_radar_maker import calculate_player_percentiles_fast
    print("✓ Successfully imported enhanced_radar_maker")
except ImportError as e:
    print(f"❌ Import failed: {e}")
    print(f"Current directory: {os.getcwd()}")
    print(f"Python path: {sys.path[-3:]}")
    
    # Alternative: define the function directly in the notebook
    def calculate_player_percentiles_fast(df, player_name, selected_metrics, sample_filter=None):
        """Simplified percentile calculation for testing"""
        from scipy.stats import percentileofscore
        
        # Get player data
        player_row = df[df['Player'] == player_name].iloc[0]
        
        # Apply sample filter
        filtered_df = df.copy()
        if sample_filter:
            if sample_filter.get('Position_Group'):
                filtered_df = filtered_df[filtered_df['Position_Group'].isin(sample_filter['Position_Group'])]
            if sample_filter.get('Minutes played'):
                filtered_df = filtered_df[filtered_df['Minutes played'] >= sample_filter['Minutes played']]
            if sample_filter.get('Competition'):
                filtered_df = filtered_df[filtered_df['Competition'].isin(sample_filter['Competition'])]
        
        percentiles = []
        for metric in selected_metrics:
            if metric in df.columns:
                player_value = player_row[metric]
                sample_values = filtered_df[metric].dropna()
                
                if len(sample_values) > 0 and not pd.isna(player_value):
                    percentile = percentileofscore(sample_values, player_value, kind='rank')
                    percentiles.append(round(percentile, 1))
                else:
                    percentiles.append(50.0)
            else:
                percentiles.append(50.0)
        
        return percentiles
    
    print("✓ Using fallback percentile calculation function")

❌ Import failed: No module named 'enhanced_radar_maker'
Current directory: e:\OneDriveFolder\OneDrive\RadarGenerator\radar-generator
Python path: ['src', './src', 'e:\\OneDriveFolder\\OneDrive\\RadarGenerator\\radar-generator\\src']
✓ Using fallback percentile calculation function


In [3]:
# Load data
df = pd.read_csv('Data/player_data_2026-02-13_113150.csv')
print(f"Data loaded: {len(df)} players, {len(df.columns)} columns")

Data loaded: 4828 players, 123 columns


In [4]:
# CONFIGURATION - Use CF preset metrics to test overlap
PLAYER_NAME = "Crysencio Summerville"  # Known high-performing player

# CF preset metrics (likely to have high percentiles = overlap issues)
TEST_METRICS = [
    "Accelerations", "Offensive Duels", "Prog. Carries",
    "Att. Aerial Duels", "Aerial Duels Won %", "Fouls Drawn", "Long Passes Received",
    "Passes Received", "Shorter Pass Acc. %", "Dribble Succ. %",
    "Succ. Def. Actions", "Att. Ground Duels",
    "NPxG", "Goals", "Shots"
]

# Sample filters
POSITION_GROUPS = ['WF', 'AM']
MIN_MINUTES = 500
COMPETITIONS = []

print(f"Testing overlap with {len(TEST_METRICS)} CF metrics")

Testing overlap with 15 CF metrics


In [5]:
# Get player data and percentiles
if PLAYER_NAME in df['Player'].values:
    player_data = df[df['Player'] == PLAYER_NAME].iloc[0]
    
    sample_filter = {
        'Position_Group': POSITION_GROUPS if POSITION_GROUPS else None,
        'Minutes played': MIN_MINUTES if MIN_MINUTES else None,
        'Competition': COMPETITIONS if COMPETITIONS else None
    }

    percentiles = calculate_player_percentiles_fast(
        df, PLAYER_NAME, TEST_METRICS, sample_filter
    )
    
    # Ensure all percentiles are floats
    percentiles = [50.0 if (pct is None or pd.isna(pct)) else float(pct) for pct in percentiles]
    
    print("Player percentiles (high values will cause overlap):")
    for metric, pct in zip(TEST_METRICS, percentiles):
        marker = "⚠️" if pct >= 90 else "✅" if pct >= 75 else ""
        print(f"{metric}: {pct}% {marker}")
        
    high_percentiles = [pct for pct in percentiles if pct >= 90]
    print(f"\n{len(high_percentiles)} metrics above 90% (likely to cause overlap)")
else:
    print("Player not found!")

Player not found!


In [6]:
# Load fonts
try:
    font_bold_prop = fm.FontProperties(fname='fonts/AlteHaasGroteskBold.ttf')
    font_regular_prop = fm.FontProperties(fname='fonts/AlteHaasGroteskRegular.ttf')
    font_bold = type('FontManager', (), {'prop': font_bold_prop})()
    font_regular = type('FontManager', (), {'prop': font_regular_prop})()
    print("✓ Loaded Alte Haas Grotesk fonts")
except:
    font_bold_prop = fm.FontProperties(weight='bold')
    font_regular_prop = fm.FontProperties(weight='normal')
    font_bold = type('FontManager', (), {'prop': font_bold_prop})()
    font_regular = type('FontManager', (), {'prop': font_regular_prop})()
    print("✓ Using default system fonts")

# Load logo
try:
    logo = Image.open('src/BLACK PNG (2).png')
    print("✓ Logo loaded")
except:
    logo = None
    print("⚠ Logo not found")

✓ Loaded Alte Haas Grotesk fonts
✓ Logo loaded


In [7]:
# OVERLAP SOLUTION FUNCTIONS

def wrap_parameter_names(params, max_length=12):
    """Wrap long parameter names onto multiple lines"""
    wrapped_params = []
    
    for param in params:
        if len(param) <= max_length:
            wrapped_params.append(param)
        else:
            words = param.replace('.', '. ').replace('%', '% ').split()
            lines = []
            current_line = ""
            
            for word in words:
                if current_line and len(current_line + " " + word) > max_length:
                    lines.append(current_line.strip())
                    current_line = word
                else:
                    if current_line:
                        current_line += " " + word
                    else:
                        current_line = word
            
            if current_line:
                lines.append(current_line.strip())
            
            wrapped_params.append("\n".join(lines))
    
    return wrapped_params

def abbreviate_metrics(params):
    """Create shorter versions of metric names"""
    abbreviations = {
        "Accelerations": "Accel",
        "Offensive Duels": "Off. Duels",
        "Prog. Carries": "Prog. Car.",
        "Att. Aerial Duels": "Aer. Duels",
        "Aerial Duels Won %": "Aer. Won %",
        "Fouls Drawn": "Fouls Dr.",
        "Long Passes Received": "Long Pass Rec.",
        "Passes Received": "Pass Rec.",
        "Shorter Pass Acc. %": "Short Acc. %",
        "Dribble Succ. %": "Drib. %",
        "Succ. Def. Actions": "Def. Act.",
        "Att. Ground Duels": "Grd. Duels"
    }
    
    return [abbreviations.get(param, param) for param in params]

def generate_colors_for_percentiles(percentiles, gradient_type='warm_to_cool'):
    """Generate colors based on percentiles"""
    gradient_colors = {
        'warm_to_cool': ['#e8a5a5', '#f4b5a5', '#f5d5a5', '#f5f5a5', '#b5d5a5', '#95d595'],
        'blue_scale': ['#ffcccc', '#cce6ff', '#99d6ff', '#66c2ff', '#3399ff', '#0066cc'],
        'purple_scale': ['#f0e6ff', '#e6ccff', '#d9b3ff', '#cc99ff', '#bf80ff', '#9933ff'],
        'ocean': ['#ffe6e6', '#e6f3ff', '#cce6ff', '#99d6ff', '#66b3ff', '#0080ff'],
        'sunset': ['#ffe6cc', '#ffcc99', '#ffb366', '#ff9933', '#ff6600', '#cc3300']
    }
    
    colors = gradient_colors.get(gradient_type, gradient_colors['warm_to_cool'])
    slice_colors = []
    
    for pct in percentiles:
        pct = float(pct) if pct is not None else 50.0
        if pct <= 10:
            slice_colors.append(colors[0])
        elif pct <= 25:
            slice_colors.append(colors[1])
        elif pct <= 50:
            slice_colors.append(colors[2])
        elif pct <= 75:
            slice_colors.append(colors[3])
        elif pct <= 90:
            slice_colors.append(colors[4])
        else:
            slice_colors.append(colors[5])
    
    return slice_colors

print("✓ Overlap solution functions loaded")

✓ Overlap solution functions loaded


In [8]:
# SOLUTION 1: ORIGINAL SETTINGS (for comparison)
print("=== SOLUTION 1: ORIGINAL SETTINGS ===")

if PLAYER_NAME in df['Player'].values:
    params_original = TEST_METRICS
    slice_colors = generate_colors_for_percentiles(percentiles)
    text_colors = ['#000000'] * len(percentiles)
    
    baker = PyPizza(
        params=params_original,
        background_color="#f5eddc",
        straight_line_color="#000000",
        straight_line_lw=0,
        last_circle_lw=1.5,
        other_circle_lw=1,
        inner_circle_size=15
    )

    fig, ax = baker.make_pizza(
        percentiles,
        figsize=(8, 9),
        param_location=110,  # Original setting
        slice_colors=slice_colors,
        value_colors=text_colors,
        value_bck_colors=slice_colors,
        blank_alpha=0.4,
        kwargs_slices=dict(edgecolor="#f5eddc", zorder=2, linewidth=4),
        kwargs_params=dict(
            color="#000000", fontsize=13, 
            fontproperties=font_bold.prop, va="center"
        ),
        kwargs_values=dict(
            color="#000000", fontsize=12, 
            fontproperties=font_bold.prop, zorder=3,
            bbox=dict(edgecolor="#000000", facecolor="cornflowerblue", 
                     boxstyle="round,pad=0.2", lw=1)
        )
    )

    plt.title("Solution 1: Original Settings (likely overlap)", pad=20)
    plt.tight_layout()
    plt.show()
    
    print("Issues: Long parameter names overlap, high percentile values overlap with params")

=== SOLUTION 1: ORIGINAL SETTINGS ===


In [9]:
# SOLUTION 2: WRAPPED PARAMETER NAMES
print("\n=== SOLUTION 2: WRAPPED PARAMETER NAMES ===")

if PLAYER_NAME in df['Player'].values:
    params_wrapped = wrap_parameter_names(TEST_METRICS, max_length=12)
    
    baker = PyPizza(
        params=params_wrapped,
        background_color="#f5eddc",
        straight_line_color="#000000",
        straight_line_lw=0,
        last_circle_lw=1.5,
        other_circle_lw=1,
        inner_circle_size=15
    )

    fig, ax = baker.make_pizza(
        percentiles,
        figsize=(8, 9),
        param_location=110,
        slice_colors=slice_colors,
        value_colors=text_colors,
        value_bck_colors=slice_colors,
        blank_alpha=0.4,
        kwargs_slices=dict(edgecolor="#f5eddc", zorder=2, linewidth=4),
        kwargs_params=dict(
            color="#000000", fontsize=10,  # Smaller font for wrapped text
            fontproperties=font_bold.prop, va="center"
        ),
        kwargs_values=dict(
            color="#000000", fontsize=12, 
            fontproperties=font_bold.prop, zorder=3,
            bbox=dict(edgecolor="#000000", facecolor="cornflowerblue", 
                     boxstyle="round,pad=0.2", lw=1)
        )
    )

    plt.title("Solution 2: Wrapped Parameter Names", pad=20)
    plt.tight_layout()
    plt.show()
    
    print("Pros: No parameter overlap. Cons: High values still overlap with wrapped params")


=== SOLUTION 2: WRAPPED PARAMETER NAMES ===


In [10]:
# SOLUTION 3: ABBREVIATED PARAMETER NAMES
print("\n=== SOLUTION 3: ABBREVIATED PARAMETER NAMES ===")

if PLAYER_NAME in df['Player'].values:
    params_abbreviated = abbreviate_metrics(TEST_METRICS)
    
    baker = PyPizza(
        params=params_abbreviated,
        background_color="#f5eddc",
        straight_line_color="#000000",
        straight_line_lw=0,
        last_circle_lw=1.5,
        other_circle_lw=1,
        inner_circle_size=15
    )

    fig, ax = baker.make_pizza(
        percentiles,
        figsize=(8, 9),
        param_location=110,
        slice_colors=slice_colors,
        value_colors=text_colors,
        value_bck_colors=slice_colors,
        blank_alpha=0.4,
        kwargs_slices=dict(edgecolor="#f5eddc", zorder=2, linewidth=4),
        kwargs_params=dict(
            color="#000000", fontsize=11,  # Can use larger font with abbreviations
            fontproperties=font_bold.prop, va="center"
        ),
        kwargs_values=dict(
            color="#000000", fontsize=12, 
            fontproperties=font_bold.prop, zorder=3,
            bbox=dict(edgecolor="#000000", facecolor="cornflowerblue", 
                     boxstyle="round,pad=0.2", lw=1)
        )
    )

    plt.title("Solution 3: Abbreviated Parameter Names", pad=20)
    plt.tight_layout()
    plt.show()
    
    print("Pros: Shorter names, less overlap. Cons: Less descriptive, high values may still overlap")


=== SOLUTION 3: ABBREVIATED PARAMETER NAMES ===


In [11]:
# SOLUTION 4: SMALLER VALUE BOXES + CLOSER TO CENTER
print("\n=== SOLUTION 4: SMALLER VALUE BOXES + CLOSER TO CENTER ===")

if PLAYER_NAME in df['Player'].values:
    baker = PyPizza(
        params=TEST_METRICS,
        background_color="#f5eddc",
        straight_line_color="#000000",
        straight_line_lw=0,
        last_circle_lw=1.5,
        other_circle_lw=1,
        inner_circle_size=15
    )

    # Try to move values closer to center by adjusting the pizza generation
    fig, ax = baker.make_pizza(
        percentiles,
        figsize=(8, 9),
        param_location=115,  # Slightly further out
        slice_colors=slice_colors,
        value_colors=text_colors,
        value_bck_colors=slice_colors,
        blank_alpha=0.4,
        kwargs_slices=dict(edgecolor="#f5eddc", zorder=2, linewidth=4),
        kwargs_params=dict(
            color="#000000", fontsize=10,  # Smaller param font
            fontproperties=font_bold.prop, va="center"
        ),
        kwargs_values=dict(
            color="#000000", fontsize=10,  # Smaller value font
            fontproperties=font_bold.prop, zorder=3,
            bbox=dict(edgecolor="#000000", facecolor="cornflowerblue", 
                     boxstyle="round,pad=0.1", lw=1)  # Smaller padding
        )
    )

    plt.title("Solution 4: Smaller Fonts + Boxes, Params Further Out", pad=20)
    plt.tight_layout()
    plt.show()
    
    print("Pros: Smaller elements reduce overlap. Cons: Less readable text")


=== SOLUTION 4: SMALLER VALUE BOXES + CLOSER TO CENTER ===


In [12]:
# SOLUTION 5: HYBRID - ABBREVIATED + OPTIMIZED SPACING
print("\n=== SOLUTION 5: HYBRID - ABBREVIATED + OPTIMIZED SPACING ===")

if PLAYER_NAME in df['Player'].values:
    params_abbreviated = abbreviate_metrics(TEST_METRICS)
    
    baker = PyPizza(
        params=params_abbreviated,
        background_color="#f5eddc",
        straight_line_color="#000000",
        straight_line_lw=0,
        last_circle_lw=1.5,
        other_circle_lw=1,
        inner_circle_size=15
    )

    fig, ax = baker.make_pizza(
        percentiles,
        figsize=(8, 9),
        param_location=115,  # Balanced distance
        slice_colors=slice_colors,
        value_colors=text_colors,
        value_bck_colors=slice_colors,
        blank_alpha=0.4,
        kwargs_slices=dict(edgecolor="#f5eddc", zorder=2, linewidth=4),
        kwargs_params=dict(
            color="#000000", fontsize=11,  # Good balance
            fontproperties=font_bold.prop, va="center"
        ),
        kwargs_values=dict(
            color="#000000", fontsize=11,  # Consistent sizing
            fontproperties=font_bold.prop, zorder=3,
            bbox=dict(edgecolor="#000000", facecolor="cornflowerblue", 
                     boxstyle="round,pad=0.15", lw=1)  # Optimized padding
        )
    )

    plt.title("Solution 5: Hybrid - Abbreviated Names + Optimized Spacing", pad=20)
    plt.tight_layout()
    plt.show()
    
    print("Pros: Best balance of readability and space efficiency")
    print("\nRECOMMENDATION: This hybrid approach seems most promising!")


=== SOLUTION 5: HYBRID - ABBREVIATED + OPTIMIZED SPACING ===


## Summary of Solutions Tested:

1. **Original Settings**: Shows the overlap problem clearly
2. **Wrapped Parameter Names**: Fixes param overlap but creates new issues with high values
3. **Abbreviated Parameter Names**: Shorter names reduce overlap significantly
4. **Smaller Elements**: Reduces overlap but hurts readability
5. **Hybrid Approach**: Combines abbreviations with optimized spacing

**Recommendation**: The hybrid approach (Solution 5) with abbreviated metric names and optimized spacing seems to provide the best balance of readability and space efficiency.

**Next Steps**: If you like one of these solutions, we can implement it in the main app!